In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Define the Generator Model
def build_generator(latent_dim):
    generator = models.Sequential()
    generator.add(layers.Dense(256, input_dim=latent_dim))
    generator.add(layers.LeakyReLU(alpha=0.2))
    generator.add(layers.BatchNormalization(momentum=0.8))
    generator.add(layers.Dense(512))
    generator.add(layers.LeakyReLU(alpha=0.2))
    generator.add(layers.BatchNormalization(momentum=0.8))
    generator.add(layers.Dense(1024))
    generator.add(layers.LeakyReLU(alpha=0.2))
    generator.add(layers.BatchNormalization(momentum=0.8))
    generator.add(layers.Dense(784, activation='tanh'))
    generator.add(layers.Reshape((28, 28, 1)))
    return generator

# Define the Discriminator Model
def build_discriminator():
    discriminator = models.Sequential()
    discriminator.add(layers.Flatten(input_shape=(28, 28, 1)))
    discriminator.add(layers.Dense(512))
    discriminator.add(layers.LeakyReLU(alpha=0.2))
    discriminator.add(layers.Dense(256))
    discriminator.add(layers.LeakyReLU(alpha=0.2))
    discriminator.add(layers.Dense(1, activation='sigmoid'))
    return discriminator

# Create the GAN Model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan = models.Sequential()
    gan.add(generator)
    gan.add(discriminator)
    return gan

# Prepare and load the dataset (MNIST in this case)
def load_mnist():
    (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    return x_train

# Training the GAN
def train_gan(gan, generator, discriminator, x_train, epochs=100, batch_size=128, latent_dim=100):
    for epoch in range(epochs):
        for _ in range(batch_size):
            noise = np.random.normal(0, 1, size=[batch_size, latent_dim])
            generated_images = generator.predict(noise)
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
            X = np.concatenate([image_batch, generated_images])
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9

            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

            noise = np.random.normal(0, 1, size=[batch_size, latent_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

        print(f"Epoch {epoch}/{epochs} | Discriminator Loss: {d_loss} | Generator Loss: {g_loss}")

# Main function to run the GAN
def main():
    latent_dim = 100
    generator = build_generator(latent_dim)
    discriminator = build_discriminator()
    gan = build_gan(generator, discriminator)
    x_train = load_mnist()

    gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=0.0002, beta_1=0.5))

    train_gan(gan, generator, discriminator, x_train)

if __name__ == "__main__":
    main()
